In [ ]:
import numpy as np
import pandas as pd
from representatives import kMeans # you can use this kMeans in Ex. 3


# Exercise 1

In [ ]:
from sklearn.decomposition import PCA

'''
  Tries to make clustering c2 equal to clustering c1 by renaming the cluster names.
  If the clusterings are effectively equivalent, the output will be equal to c1.
'''
def try_unification(c1, c2):
    v1 = list(np.unique(c1))
    v2 = list(np.unique(c2))
    new_vals = []
    if len(c1) != len(c2):
        print("Cannot unify clusterings of different lengths!")
        return None
    if len(v1) != len(v2):
        print("Cannot unify clusterings of different numbers of clusters!")
        return None
    
    
    # use different symbols for clusterings
    i = 0
    for v in v2:
        c = "v" + str(i)
        while c in v1:
            i +=1
            c = "v" + str(i)
        new_vals.append(c)
        i += 1
    c_new = [new_vals[v2.index(i)] for i in c2]
    
    # replace occurrences
    targets = []
    for symbol in new_vals:
        first_index = c_new.index(symbol)
        replace_symbol = None
        for v in c1:
            if not v in targets:
                replace_symbol = v
                break
        if replace_symbol in targets:
            print("Warning: No unification possible, the symbol " + replace_symbol + " has already been addressed before!")
            return None
        
        c_new = [replace_symbol if v == symbol else v for v in c_new]
        targets.append(replace_symbol)
    return c_new

# Test Complete Link on Iris (PCA)
dfIris = pd.read_csv("iris.csv")
DIris = PCA(n_components=2).fit_transform(dfIris.values[:,:4])
labelsIris = list(pd.unique(dfIris["species"]))
C_perfect = np.array([labelsIris.index(l) for l in dfIris["species"]])
C_iris = np.array(try_unification(C_perfect, agglomerativeClustering(DIris, completeLink, k=3)))
M_expected = np.array([[50, 0, 0], [0, 14, 49], [0, 36, 1]]) # according to slide 13
for i in range(3):
    cond1 = C_iris == i
    for j in range(3):
        cond2 = C_perfect == j
        cnt_combo = np.count_nonzero(cond1 & cond2)
        print(labelsIris[i] + "/" + labelsIris[j] + ": " + ("OK" if cnt_combo == M_expected[i,j] else "FAILED. Expected " + str(M_expected[i,j]) + " but saw " + str(cnt_combo)))

# Test Coincidence of Standard and Lance-Williams
for D in [DIris, pd.get_dummies(pd.read_csv("Mall_Customers.csv")).values]:
    for pair in [("single", singleLink), ("complete", completeLink), ("groupavg", groupAverage), ("meandist", meanDistance), ("ward", ward)]:
        C_ac = np.array(agglomerativeClustering(D, pair[1], k=3))
        C_aclw = np.array(try_unification(C_ac, agglomerativeClusteringLW(D, pair[0], k=3)))
        mismatches = C_ac != C_aclw
        print(pair[0] + (" OK" if np.count_nonzero(mismatches) == 0 else (" FAILED. Difference in positions: " + str(np.where(mismatches)[0]) + ": " + str(C_ac[mismatches]) + " vs " + str(C_aclw[mismatches]))))